# Analyzing borrowers’ risk of defaulting project.

Hello, this is a report for our bank's loan division.
In this report I will attempt to **find out if a customer’s marital status and number of children has an impact on whether they will default on a loan.**

In [1]:
# importing libraries and reading the data

import pandas as pd
import warnings
warnings.filterwarnings("ignore")
try:
    df= pd.read_csv('credit_scoring_eng.csv')
except: 
    df= pd.read_csv('/datasets/credit_scoring_eng.csv')

## Task 1. Data exploration

**Description of the data**
- `children` - the number of children in the family
- `days_employed` - work experience in days
- `dob_years` - client's age in years
- `education` - client's education
- `education_id` - education identifier
- `family_status` - marital status
- `family_status_id` - marital status identifier
- `gender` - gender of the client
- `income_type` - type of employment
- `debt` - was there any debt on loan repayment
- `total_income` - monthly income
- `purpose` - the purpose of obtaining a loan

Now let's explore our data and look at a few rows to check for potential issues with the data.

In [2]:
# let's print the first 10 rows

df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


Something is wrong with 'days_employed' column, it's values are not realistic. Let's have a closer look at the rows with missing data using .info() method.

In [3]:
# Get info on data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


We can see that we have missing values in two of our columns, which are 'days_employed' and 'total_income'. Also as of note both of them are in float data type which is greatly confusing, since other numerical columns are just integers.
Let's have a look at missing values in both of our columns - 'days_employed' and 'total_income'. I can use indexing to help narrowing our search.

In [4]:
df[df['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


While only showing rows with missing data in column 'days_emloyed', we can clearly see that column 'total_income' has them as well. Let's print out the 'total_income' column and both of them at the same time. 

In [5]:
df[df['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [6]:
# Let's apply multiple conditions for filtering data and look at the number of rows in the filtered table.

df[(df['total_income'].isna()) & (df['days_employed'].isna())]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


As we can see number of missing values seems to be the same for both our columns.

Let's calculate the percentages of the missing values in our columns with total ammount of values.

In [7]:
# Displaying a percentile of missing data in 'days_employed' column

(df['days_employed'].isna().sum()) / (df['days_employed'].count()) 

0.11234561521368405

In [8]:
# Displaying a percentile of missing data in 'total_income' column

(df['total_income'].isna().sum()) / (df['total_income'].count()) 

0.11234561521368405

In [9]:
# Percentage of missing values in one column out of every row

len(df.index) / (df['days_employed'].isna().sum())

9.901103955841766

Now let's use .groupby() method supported with .count() to help us visualize our data.

In [10]:
# Displaying the distribution of missing values in our data by income type

df.groupby(by=['income_type']).count()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,debt,total_income,purpose
income_type,,,,,,,,,,,
business,5085,4577,5085,5085,5085,5085,5085,5085,5085,4577,5085
civil servant,1459,1312,1459,1459,1459,1459,1459,1459,1459,1312,1459
employee,11119,10014,11119,11119,11119,11119,11119,11119,11119,10014,11119
entrepreneur,2,1,2,2,2,2,2,2,2,1,2
paternity / maternity leave,1,1,1,1,1,1,1,1,1,1,1
retiree,3856,3443,3856,3856,3856,3856,3856,3856,3856,3443,3856
student,1,1,1,1,1,1,1,1,1,1,1
unemployed,2,2,2,2,2,2,2,2,2,2,2


We can clearly see that only 'business', 'civil servant', 'employee', 'entrepreneur' and 'retiree' have missing values. All of them are most populated columns in our dataframe except 'entrepreneur'. 

Looking at the data we could order income types by ammount of values missing:
1. employee
2. business
3. retiree
4. civil servant
5. entrepreneur

In [11]:
# Checking distribution of missing values

df[(df['days_employed'].isna())&(df['total_income'].isna())]['income_type'].value_counts()

employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64

**Intermediate conclusion**

About 20% of values in our dataframe are missing and the values in 'days_employed' which are not missing do not seem to correlate with reality. Also 2 of our columns which contain missing values are in float dtype.


**Possible reasons for missing values in data**

All this could be due to manual input error. And so we have to decide which values are not not be possible to restore and which still could be saved.

Are missing values truly random?

In [12]:
# Checking the distribution in the whole dataset

df['income_type'].value_counts()

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

**The distribution in the original dataset similar to the distribution of the filtered table.**

**Conclusions**

The more popular groups of income type tend to have more missing values with regular employees at the top.
This makes sense because they are the absolute majority of our world in terms of sheer count. Missing values are inevitable to come by especially if data's being inputed manually. 

I decided to get rid of 'days_employed' column since it's values contain negative and unrealistic values. This is unfortunate but it's badly damaged.

We can try to fill missing values in 'total_income' column with average value per each group. Also it's still in float dtype so we need to convert it to integer.

We also still need to check for duplicates in our data.


## Data transformation

As I've decided let's remove the 'days_employed' column entirely.

After let's check for duplicates and try fixing the values in 'education' column.

In [13]:
# Dropping column

df = df.drop(columns=['days_employed'])
df

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...
21520,1,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


In [14]:
# Showing only duplicated data

df[df.duplicated()]

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,41,secondary education,1,married,0,F,employee,0,NaN,purchase of the house for my family
4182,1,34,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,NaN,wedding ceremony
4851,0,60,secondary education,1,civil partnership,1,F,retiree,0,NaN,wedding ceremony
5557,0,58,secondary education,1,civil partnership,1,F,retiree,0,NaN,to have a wedding
7808,0,57,secondary education,1,civil partnership,1,F,retiree,0,NaN,having a wedding
8583,0,58,bachelor's degree,0,unmarried,4,F,retiree,0,NaN,supplementary education
9238,2,34,secondary education,1,married,0,F,employee,0,NaN,buying property for renting out
9528,0,66,secondary education,1,widow / widower,2,F,retiree,0,NaN,transactions with my real estate
9627,0,56,secondary education,1,married,0,F,retiree,0,NaN,transactions with my real estate
10462,0,62,secondary education,1,married,0,F,retiree,0,NaN,buy commercial real estate


In [15]:
df.duplicated().sum()

54

We have summed up all known duplicates and we have 54 duplicates. Let's confirm that all the values in 'total_income' are missing.

In [16]:
df[df.duplicated()].groupby(by=['education']).count()

,children,dob_years,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
education,,,,,,,,,,
BACHELOR'S DEGREE,1,1,1,1,1,1,1,1,0,1
bachelor's degree,8,8,8,8,8,8,8,8,0,8
secondary education,45,45,45,45,45,45,45,45,0,45


So by grouping up only duplicated data in our dataset and doing simple count to check for missing data we noticed an interesting detail. Let's take a note of that.

In [17]:
# Let's see all values in education column to check if and what spellings will need to be fixed

df['education'].unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

This is quite a few implicit duplicates. Let's use the str.lower() method and resave the values.

In [18]:
# Fix the registers if required

df['education'] = df['education'].str.lower()

In [19]:
# Checking all the values in the column to make sure we fixed them

df['education'].unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

In [20]:
# Let's try checking for duplicates once again

df.duplicated().sum()

71

There's more duplicates in our data, how curious.

In [21]:
df = df.drop_duplicates()

In [22]:
df.duplicated().sum()

0

In [23]:
# Let's see the distribution of values in the `children` column

df['children'].value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [24]:
df.groupby(by=['children']).count()

,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
children,,,,,,,,,,
-1,47,47,47,47,47,47,47,47,44,47
0,14091,14091,14091,14091,14091,14091,14091,14091,12710,14091
1,4808,4808,4808,4808,4808,4808,4808,4808,4343,4808
2,2052,2052,2052,2052,2052,2052,2052,2052,1851,2052
3,330,330,330,330,330,330,330,330,294,330
4,41,41,41,41,41,41,41,41,34,41
5,9,9,9,9,9,9,9,9,8,9
20,76,76,76,76,76,76,76,76,67,76


Having 20 children must be quite hard, maybe as hard as having -1 child.

In [25]:
# Let's replace rows with values '-1' and '20' to '0'

df['children'] = df['children'].replace([-1, 20], 0)

In [26]:
# Checking the `children` column again to make sure it's all fixed

df.groupby(by=['children']).count()

,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
children,,,,,,,,,,
0,14214,14214,14214,14214,14214,14214,14214,14214,12821,14214
1,4808,4808,4808,4808,4808,4808,4808,4808,4343,4808
2,2052,2052,2052,2052,2052,2052,2052,2052,1851,2052
3,330,330,330,330,330,330,330,330,294,330
4,41,41,41,41,41,41,41,41,34,41
5,9,9,9,9,9,9,9,9,8,9


In [27]:
# Checking the `dob_years` for suspicious values and count the percentage

df.groupby(by=['dob_years']).count()

,children,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
dob_years,,,,,,,,,,
0,101,101,101,101,101,101,101,101,91,101
19,14,14,14,14,14,14,14,14,13,14
20,51,51,51,51,51,51,51,51,46,51
21,111,111,111,111,111,111,111,111,93,111
22,183,183,183,183,183,183,183,183,166,183
23,252,252,252,252,252,252,252,252,218,252
24,264,264,264,264,264,264,264,264,243,264
25,357,357,357,357,357,357,357,357,334,357
26,408,408,408,408,408,408,408,408,373,408


Here we have entries with 0 years, which don't make sense at all. I'm not really sure how one could handle these rows other than deleting them out right. 

In [28]:
# Resaving data without 0 value rows in column

df = df[df['dob_years'] > 0]

In [29]:
# Checking the result - make sure it's fixed

df.groupby(by=['dob_years']).count()

,children,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
dob_years,,,,,,,,,,
19,14,14,14,14,14,14,14,14,13,14
20,51,51,51,51,51,51,51,51,46,51
21,111,111,111,111,111,111,111,111,93,111
22,183,183,183,183,183,183,183,183,166,183
23,252,252,252,252,252,252,252,252,218,252
24,264,264,264,264,264,264,264,264,243,264
25,357,357,357,357,357,357,357,357,334,357
26,408,408,408,408,408,408,408,408,373,408
27,493,493,493,493,493,493,493,493,457,493


Onto the next column, let's check 'family_status' column.

In [30]:
# Let's see the values for the column

df.groupby(by=['family_status']).count()

,children,dob_years,education,education_id,family_status_id,gender,income_type,debt,total_income,purpose
family_status,,,,,,,,,,
civil partnership,4130,4130,4130,4130,4130,4130,4130,4130,3717,4130
divorced,1185,1185,1185,1185,1185,1185,1185,1185,1074,1185
married,12290,12290,12290,12290,12290,12290,12290,12290,11098,12290
unmarried,2794,2794,2794,2794,2794,2794,2794,2794,2510,2794
widow / widower,954,954,954,954,954,954,954,954,861,954


Everythings fine bar some missing values in 'total_income'.

Onto 'gender' column.

In [31]:
# Let's see the values in the column

df.groupby(by=['gender']).count()

,children,dob_years,education,education_id,family_status,family_status_id,income_type,debt,total_income,purpose
gender,,,,,,,,,,
F,14102,14102,14102,14102,14102,14102,14102,14102,12688,14102
M,7250,7250,7250,7250,7250,7250,7250,7250,6571,7250
XNA,1,1,1,1,1,1,1,1,1,1


One of the rows in our data does not seem to represent one of two provided genders. Of course we could leave it be since it doesn't have much influence on our research.

In [32]:
# Let's see the values in the column

df.groupby(by=['income_type']).count()

,children,dob_years,education,education_id,family_status,family_status_id,gender,debt,total_income,purpose
income_type,,,,,,,,,,
business,5058,5058,5058,5058,5058,5058,5058,5058,4559,5058
civil servant,1451,1451,1451,1451,1451,1451,1451,1451,1306,1451
employee,11029,11029,11029,11029,11029,11029,11029,11029,9964,11029
entrepreneur,2,2,2,2,2,2,2,2,1,2
paternity / maternity leave,1,1,1,1,1,1,1,1,1,1
retiree,3809,3809,3809,3809,3809,3809,3809,3809,3426,3809
student,1,1,1,1,1,1,1,1,1,1
unemployed,2,2,2,2,2,2,2,2,2,2


Some of the values seem to be implicit duplicates. 'Student' can be considered as 'unemployed' same as 'paternity / maternity leave' and 'entrepreneur'. 

But let's leave it as it is for now.

# Working with missing values

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21353 entries, 0 to 21524
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21353 non-null  int64  
 1   dob_years         21353 non-null  int64  
 2   education         21353 non-null  object 
 3   education_id      21353 non-null  int64  
 4   family_status     21353 non-null  object 
 5   family_status_id  21353 non-null  int64  
 6   gender            21353 non-null  object 
 7   income_type       21353 non-null  object 
 8   debt              21353 non-null  int64  
 9   total_income      19260 non-null  float64
 10  purpose           21353 non-null  object 
dtypes: float64(1), int64(5), object(5)
memory usage: 2.0+ MB


We have some columns which could be converted to dictionaries:
- 'education' with 'education_id'
- 'family_status' with 'family_status_id'.


In [34]:
dict(zip(df["education_id"], df["education"]))

{0: "bachelor's degree",
 1: 'secondary education',
 2: 'some college',
 3: 'primary education',
 4: 'graduate degree'}

In [35]:
dict(zip(df["family_status_id"], df["family_status"]))

{0: 'married',
 1: 'civil partnership',
 2: 'widow / widower',
 3: 'divorced',
 4: 'unmarried'}

### Restoring missing values in `total_income`

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21353 entries, 0 to 21524
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21353 non-null  int64  
 1   dob_years         21353 non-null  int64  
 2   education         21353 non-null  object 
 3   education_id      21353 non-null  int64  
 4   family_status     21353 non-null  object 
 5   family_status_id  21353 non-null  int64  
 6   gender            21353 non-null  object 
 7   income_type       21353 non-null  object 
 8   debt              21353 non-null  int64  
 9   total_income      19260 non-null  float64
 10  purpose           21353 non-null  object 
dtypes: float64(1), int64(5), object(5)
memory usage: 2.0+ MB


To help us gain more information on our data, we want to know what is the percentage of missing data to our whole dataset.

In [37]:
# Displaying a percentage of missing values

df['total_income'].isna().sum() / len(df.index) 

0.09801901372172528

This is a significant ammount of data, so should try replacing the missing values with something believable to help us archive our goal in testing our hypothesis on whether person would **default a loan or not** based on them **having kids**, their **type of income** or **martial status**.

Let's adress missing values in 'total_income'. 
To do that we need to find appropriate values for each age group. 

Let's create a function to help us do that.

In [38]:
# Let's write a function that calculates the age category

def age_category_clients(age):
    
    if age <= 29:
        return 'young'
    if age <= 49:
        return 'mature'
    if age <= 69:
        return 'adult'
    if age >= 70:
        return 'old'
    return 'out of range'
        

Our age groups are as follows:
1. **Young** if less than 29
2. **Mature** if 30 or less than 49
3. **Adult** if 50 or less than 69
4. **Old** for greater than 70

In [39]:
# Test if the function works

age_category_clients(55)

'adult'

It does, let's .apply() it to a new column. 

In [40]:
# Creating new column based on function

df['age_category'] = df['dob_years'].apply(age_category_clients)

In [41]:
# Doing a count of values in new column

df['age_category'].value_counts()

mature    11016
adult      6988
young      3180
old         169
Name: age_category, dtype: int64

Doing a .value_counts() method to evaluate how big each of the groups is, here we see that people ages of 30 to 49 take the majority of our data. 


To help us get the right data for replacement let's create a new dataset without rows with missing values.

We'll be using .dropna() method and resaving it to 'clean_df'.

In [42]:
# Creating a table without missing values 

clean_df = df.dropna()

clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19260 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          19260 non-null  int64  
 1   dob_years         19260 non-null  int64  
 2   education         19260 non-null  object 
 3   education_id      19260 non-null  int64  
 4   family_status     19260 non-null  object 
 5   family_status_id  19260 non-null  int64  
 6   gender            19260 non-null  object 
 7   income_type       19260 non-null  object 
 8   debt              19260 non-null  int64  
 9   total_income      19260 non-null  float64
 10  purpose           19260 non-null  object 
 11  age_category      19260 non-null  object 
dtypes: float64(1), int64(5), object(6)
memory usage: 1.9+ MB


In [43]:
# Printing a few rows

clean_df.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,mature
1,1,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,mature
2,0,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,mature
3,3,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,mature
4,0,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,adult


Now we have to decide are we going to use mean or median values for replacement.

We can use pivot tables to help us with that.

In [44]:
# Looking at the mean values for income based on your identified factors

table_age_means = pd.pivot_table(clean_df, index='age_category', values='total_income')
table_age_means

,total_income
age_category,
adult,24953.766431
mature,28428.624153
old,20125.658331
young,25533.960641


In [45]:
# Looking at the median values for income based on your identified factors

table_age_medians = pd.pivot_table(clean_df, index='age_category', values='total_income', aggfunc='median')
table_age_medians

,total_income
age_category,
adult,21437.0100
mature,24722.2370
old,18751.3240
young,22742.6535


Now we know general income bases for our age groups, but what if we include 'income_type' column in the same pivot table?

Let's take a look.

In [46]:
# Using pivot table to display the mean values in age groups for income groups

table_income_mean = pd.pivot_table(clean_df, index=['income_type', 'age_category'], values='total_income')
table_income_mean

total_income
income_type                 age_category              
business                    adult         32402.629191
                            mature        33545.590108
                            old           27766.307200
                            young         28690.055945
civil servant               adult         26467.358669
                            mature        28241.054300
                            old           32189.795667
                            young         25145.840248
employee                    adult         26246.723256
                            mature        26192.783317
                            old           26672.382429
                            young         24099.859203
entrepreneur                young         79866.103000
paternity / maternity leave mature         8612.661000
retiree                     adult         21869.540831
                            mature        26295.606609
                            old           18994.044264
                            young         14888.651857
student                     young         15712.260000
unemployed                  mature        21014.360500

In [47]:
# Displaying median values for same groups

table_income_median = pd.pivot_table(clean_df, index=['income_type', 'age_category'], values='total_income', aggfunc='median')
table_income_median

total_income
income_type                 age_category              
business                    adult           27462.2990
                            mature          28729.9420
                            old             28138.8950
                            young           25570.8550
civil servant               adult           23158.6890
                            mature          24708.8285
                            old             24525.2240
                            young           23504.5105
employee                    adult           22688.1850
                            mature          23156.8480
                            old             24660.9010
                            young           21537.4790
entrepreneur                young           79866.1030
paternity / maternity leave mature           8612.6610
retiree                     adult           18835.6000
                            mature          22118.2345
                            old             17650.4660
                            young           12807.0710
student                     young           15712.2600
unemployed                  mature          21014.3605

We should use **median** method for this case because it makes our data more transcriptive.

Now we can use .fillna() method with .transform() to fill missing values.

In [48]:
#  Filling missing values in 'total_income' by grouping our data and transforming the mean
        
df['total_income'] = df['total_income'].fillna(df.groupby(['age_category', 'income_type'])['total_income'].transform('mean'))

Let's see if we have any missing values in our data.

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21353 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21353 non-null  int64  
 1   dob_years         21353 non-null  int64  
 2   education         21353 non-null  object 
 3   education_id      21353 non-null  int64  
 4   family_status     21353 non-null  object 
 5   family_status_id  21353 non-null  int64  
 6   gender            21353 non-null  object 
 7   income_type       21353 non-null  object 
 8   debt              21353 non-null  int64  
 9   total_income      21352 non-null  float64
 10  purpose           21353 non-null  object 
 11  age_category      21353 non-null  object 
dtypes: float64(1), int64(5), object(6)
memory usage: 2.1+ MB


## Categorization of data

At this point we should look at the 'purpose' column. As we'll see it has a lot of implicit duplicates that we should categorize.


In [50]:
# Printing the values for selected data for categorization

df['purpose'].value_counts()

wedding ceremony                            786
having a wedding                            764
to have a wedding                           760
real estate transactions                    672
buy commercial real estate                  658
buying property for renting out             649
transactions with commercial real estate    648
housing transactions                        646
purchase of the house                       640
housing                                     640
purchase of the house for my family         637
construction of own property                633
property                                    629
transactions with my real estate            627
building a real estate                      621
building a property                         619
purchase of my own house                    619
buy real estate                             618
housing renovation                          605
buy residential real estate                 603
buying my own car                       

Here we see all the values counted up. We can also look all the unique values

In [51]:
# Checking the unique values

df['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

We can identify a few groups in our list:
- car related
- house related
- wedding related
- education related

Let's write a function that we can use to create a new column in our data.

In [52]:
# Let's write a function to categorize the data based on common topics

def categorize(purpose):
    new_list = purpose.split(' ')
    
    if 'house' in new_list or 'housing' in new_list:
        return 'house'
    
    elif 'car' in new_list:
        return 'car'
    
    elif 'wedding' in new_list:
        return 'wedding'
    
    elif 'education' in new_list or 'educated' in new_list:
        return 'education'
    
    elif 'university' in new_list:
        return 'education'
    
    elif 'estate' in new_list:
        return 'house'
    
    elif 'property' in new_list:
        return 'house'
    
    else:
        return 'other'

In [53]:
# Applying our function to test if it works

df['purpose'].apply(categorize)

0            house
1              car
2            house
3        education
4          wedding
           ...    
21520        house
21521          car
21522        house
21523          car
21524          car
Name: purpose, Length: 21353, dtype: object

In [54]:
# Creating a new column with the function

df['clean_purp'] = df['purpose'].apply(categorize)

In [55]:
# Displying count for values in column

df['clean_purp'].value_counts()

house        10764
education     3995
car           3808
wedding       2310
other          476
Name: clean_purp, dtype: int64

In [56]:
# Getting summary statistics for the column

df['total_income'].describe()

count     21352.000000
mean      26797.191055
std       15708.036027
min        3306.762000
25%       17222.623000
50%       24099.859203
75%       32290.550750
max      362496.645000
Name: total_income, dtype: float64

To decide what ranges to use in categorizing let's look at the summary statistics for our 'total_income' column.

Let's use 25% range for lower income, 50% for middle income and 75% for higer income.

In [57]:
# Creating a new column using qcut method from pandas

df['category'] = pd.qcut(df['total_income'], 
                         q=5, 
                         labels=['low income', 'lower income', 'middle income', 'higher income', 'high income'])

In [58]:
# Counting each categories values to see the distribution to make sure it's divided equally

df['category'].value_counts()

middle income    4412
low income       4271
high income      4271
lower income     4270
higher income    4128
Name: category, dtype: int64

Now let's finally test our hypotheses.

## Checking the Hypotheses


**Is there a correlation between having children and paying back on time?**

In [59]:
# Getting total ammount of parents with debt

df.groupby(by='children')['debt'].count()

children
0    14144
1     4792
2     2039
3      328
4       41
5        9
Name: debt, dtype: int64

In [60]:
# Getting the summary of debted

df.groupby(by='children')['debt'].sum()

children
0    1067
1     441
2     194
3      27
4       4
5       0
Name: debt, dtype: int64

In [61]:
# Showing the loan defaulting percenteges per child

df.groupby(by='children')['debt'].mean().sort_values(ascending=True)

children
5    0.000000
0    0.075438
3    0.082317
1    0.092028
2    0.095145
4    0.097561
Name: debt, dtype: float64

**Conclusion**

Here we see that families with 2 and 4 children have higher percentages of default than the rest. That draws an answer to the question that having children 1-4 does have an impact on repaying loan on time.

**Is there a correlation between family status and paying back on time?**

In [62]:
# Getting total ammount

df.groupby(by='family_status')['debt'].count()

family_status
civil partnership     4130
divorced              1185
married              12290
unmarried             2794
widow / widower        954
Name: debt, dtype: int64

In [63]:
# Getting the summary of debted

df.groupby(by='family_status')['debt'].sum()

family_status
civil partnership    386
divorced              85
married              927
unmarried            273
widow / widower       62
Name: debt, dtype: int64

In [64]:
# Showing the loan defaulting percenteges per group

df.groupby(by='family_status')['debt'].sum() / df.groupby(by='family_status')['debt'].count()

family_status
civil partnership    0.093462
divorced             0.071730
married              0.075427
unmarried            0.097709
widow / widower      0.064990
Name: debt, dtype: float64

**Conclusion**

Here we see that divorced, married and widowed people are more likely to not default on a loan. So our most debted groups are people in civil partnership and unmarried. This is true since the latter groups are generally not as financially stable as the prior groups.

**Is there a correlation between income level and paying back on time?**

In [65]:
# Getting total ammount

df.groupby(by='income_type')['debt'].count()

income_type
business                        5058
civil servant                   1451
employee                       11029
entrepreneur                       2
paternity / maternity leave        1
retiree                         3809
student                            1
unemployed                         2
Name: debt, dtype: int64

In [66]:
# Getting summary of debted

df.groupby(by='income_type')['debt'].sum()

income_type
business                        375
civil servant                    86
employee                       1055
entrepreneur                      0
paternity / maternity leave       1
retiree                         215
student                           0
unemployed                        1
Name: debt, dtype: int64

In [67]:
# Showing the loan defualting percentages per group

df.groupby(by='category')['debt'].sum() / df.groupby(by='category')['debt'].count()

category
low income       0.080309
lower income     0.082670
middle income    0.089302
higher income    0.083333
high income      0.070007
Name: debt, dtype: float64

**Conclusion**

Here we see that employee and paternity / maternity leave groups have the highest percentages of not repaying loan on time. This makes sense since workers are the most oppressed group in our society thus them having most trouble in financial stability.

**How does credit purpose affect the default rate?**

In [68]:
# Checking the percentages for default rate for each credit purpose

df.groupby(by='clean_purp')['debt'].mean()

clean_purp
car          0.093487
education    0.092616
house        0.072371
other        0.092437
wedding      0.079654
Name: debt, dtype: float64

In [69]:
df['clean_purp'].value_counts()

house        10764
education     3995
car           3808
wedding       2310
other          476
Name: clean_purp, dtype: int64

**Conclusion**

Apperantly, buying / renting a car or getting education have the highest rate of defaulting. This is mostly due to the fact that in US, which this data is most likely based in, the social economic system is set up in the way that defaulting for those two purposes doesn't lead to poverty right away but is relatively safe, thus such high ammount of defaulting.


# General Conclusion 

Our dataset wasn't a perfect one. It had missing values in 2 columns, it had duplictes, one of the columns we even had to drop due to it being filled with unusable data. After many hours of studying and fixing problems with the data it was possible for me to draw several conclusions from it.

Also we have tested the hypotheses: 
- Having children does impact defaulting on a loan;
- Having a martial status does impact defaulting on a loan;
- Having simple job has the highest debt rate;
- Some loan purposes default more regular than others.